In [1]:
CREATE DATABASE DRM_Project;
USE drm_project;
create table rfm_calculation as
WITH RFM_Data as (
    SELECT
        cr.id,
        DATEDIFF('2022-09-01', MAX(SK_TR_TO_DATE(t.purchase_date, '%m/%d/%Y'))) AS recency,
        ROUND(1.00 * COUNT(t.purchase_date) / (DATEDIFF('2022-09-01', STR_TO_DATE(cr.created_date, '%m/%d/%Y'))/365), 2) AS frequency,
        ROUND(1.00 * SUM(t.ctv) / (DATEDIFF('2022-09-01', STR_TO_DATE(cr.created_date, '%m/%d/%Y'))/365), 0) AS monetary,
        FROM drm_project.customer_registered cr
        LEFT JOIN drm_project.customer_transaction ct ON ct.customerid = cr.id
    GROUP BY cr.id
)
select * ,
    row_number() OVER (order by recency) as rn_recency,
    row_number() OVER (order by frequency) as rn_frequency,
    row_number() OVER (order by monetary) as rn_monetary
from RFM_Data;
use drm_project;
create table rfm_final as
WITH RFM_Segments as (SELECT
    *,
    CASE
        WHEN recency <= (select round(count(customerID) * 0.25, 0) from drm_project.rfm_calculation) then 1
        WHEN recency <= (select round(count(customerID) * 0.50, 0) from drm_project.rfm_calculation) then 2
        WHEN recency <= (select round(count(customerID) * 0.75, 0) from drm_project.rfm_calculation) then 3
        ELSE 4 as rn_recency
    END as rn_recency,
    CASE
        WHEN frequency > (select round(count(customerID) * 0.75, 0) from drm_project.rfm_calculation) then 1
        WHEN frequency > (select round(count(customerID) * 0.50, 0) from drm_project.rfm_calculation) then 2
        WHEN frequency > (select round(count(customerID) * 0.25, 0) from drm_project.rfm_calculation) then 3
        ELSE 4 as rn_frequency
    END as rn_frequency,
    CASE
        WHEN monetary > (select round(count(customerID) * 0.75, 0) from drm_project.rfm_calculation) then 1
        WHEN monetary > (select round(count(customerID) * 0.50, 0) from drm_project.rfm_calculation) then 2
        WHEN monetary > (select round(count(customerID) * 0.25, 0) from drm_project.rfm_calculation) then 3
        ELSE 4 as rn_monetary
    END as rn_monetary
FROM drm_project.rfm_calculation)
select *,
    CASE
        WHEN rn_r = 1 AND rn_f = 3 AND rn_m = 4 THEN 'Best'
        WHEN rn_r IN (2,3) AND rn_f IN (3,4) AND rn_m IN (4,5) THEN 'Loyal'
        WHEN rn_r IN (3,4) AND rn_f IN (1,2) AND rn_m IN (2,3) THEN 'Promising'
        WHEN rn_r IN (4,5) AND rn_f IN (1,2) AND rn_m IN (1,2) THEN 'At Risk'
        ELSE 'Other' as RFM_Matrix
    from drm_project.rfm_final;


SyntaxError: invalid syntax (<ipython-input-1-902cde3ac427>, line 1)